In [19]:
# IMPORT DES LIBRAIRIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy.stats as st
import re

### READ FILE

In [20]:
df = pd.read_excel("./4 - Salaries_Data.xlsx")

### CLONING DATA

In [21]:
data = df

### DATA TYPES

In [22]:
# WHAT ARE THE DIFFERENT TYPES OF DATA IN OUR DATAFRAME :
display(data.dtypes.to_frame())

,0
timestamp,object
company,object
level,object
title,object
totalyearlycompensation,float64
location,object
yearsofexperience,float64
yearsatcompany,float64
tag,object
basesalary,float64


### NEW COLUMNS TO WORK WITH


In [23]:
# CREATING NEW COMPANY & TITLE COLUMN : to add new company column and delete the existing one

data['company_name'] = data['company'].str.upper()
data['title_new'] = data["title"].str.replace(r'Soft Engineer', 'Software Engineer', regex=True)

data.head(2)

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education,company_name,title_new
0,7/2/2019 16:43:16,Intel,Grade 7,Hardware Engineer,200000.0,"Santa Clara, CA",14.0,3.0,Physical Design,163000.0,...,0.0,1.0,0.0,0.0,0.0,0.0,Asian,NaN,INTEL,Hardware Engineer
1,10/13/2019 11:43:20,Latch,Manager,Software Engineering Manager,170000.0,"New York, NY",6.0,2.0,Full Stack,170000.0,...,0.0,1.0,0.0,0.0,0.0,0.0,Asian,NaN,LATCH,Software Engineering Manager


### DROPPING

In [24]:
# DROPPING IRRELEVANT COLUMNS OR COLUMNS WE ARE NOT USING :

dropped_data = data.drop(["title", "timestamp", "company", "Race_Asian", "Race_White", "Race_Two_Or_More","Race_Black","Race_Hispanic",
                          "Masters_Degree", "Bachelors_Degree", "Doctorate_Degree", "Highschool", "Some_College", "level"], axis=1)

dropped_data.head(2)

,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber,Race,Education,company_name,title_new
0,200000.0,"Santa Clara, CA",14.0,3.0,Physical Design,163000.0,10000.0,27000.0,Female,Asian,7434,807.0,11381,Asian,NaN,INTEL,Hardware Engineer
1,170000.0,"New York, NY",6.0,2.0,Full Stack,170000.0,0.0,0.0,Male,Asian,10182,501.0,16214,Asian,NaN,LATCH,Software Engineering Manager


### RENAMING COLUMNS

In [25]:
# RENAMING COLUMN NAMES FOR MORE CLARITY :

dropped_data.rename(columns = {'totalyearlycompensation': 'total_yearly_compensation', 'yearsofexperience': 'years_of_experience', 'yearsatcompany': 'years_at_company',
                       'basesalary': 'base_salary', 'stockgrantvalue': 'stock_grant_value', 'cityid': 'city_id','rowNumber': 'row_number', 'dmaid' : 'dma_id', 'Race' : 'race', 'Education' : 'education', 'title_new' : 'title'}, inplace=True)


### REORDERING COLUMNS

In [26]:
# REORDERING OUR DATA FRAME COLUMNS TO MAKE IT MORE READABLE/EXPLOITABLE :

new_columns_order = ["row_number", "company_name", "location", "city_id", "dma_id", "years_of_experience", "years_at_company", "title", "tag", "total_yearly_compensation", "base_salary", "stock_grant_value", "bonus",
                     "gender", "race", "education", "otherdetails"]
reordered_data = dropped_data[new_columns_order]

reordered_data.head(2)


,row_number,company_name,location,city_id,dma_id,years_of_experience,years_at_company,title,tag,total_yearly_compensation,base_salary,stock_grant_value,bonus,gender,race,education,otherdetails
0,11381,INTEL,"Santa Clara, CA",7434,807.0,14.0,3.0,Hardware Engineer,Physical Design,200000.0,163000.0,10000.0,27000.0,Female,Asian,NaN,Asian
1,16214,LATCH,"New York, NY",10182,501.0,6.0,2.0,Software Engineering Manager,Full Stack,170000.0,170000.0,0.0,0.0,Male,Asian,NaN,Asian


### LOCATING MISSING DATA

In [27]:
# # OÙ SONT LOCALISEES LES DONNEES MANQUANTES DANS NOTRE DATAFRAME ?

reordered_data.isnull().sum()

# salary_is_null = data.loc[data['base_salary'].isnull(),:]
# salary_is_null.head(10)


row_number                       0
company_name                     5
location                         3
city_id                          0
dma_id                           2
years_of_experience              0
years_at_company                 0
title                            3
tag                            854
total_yearly_compensation        3
base_salary                      1
stock_grant_value                3
bonus                            1
gender                       19540
race                         40215
education                    32272
otherdetails                 22505
dtype: int64

### DROPPING MISSING DATA

In [28]:
# MISSING VALUES ARE LESS THAN 2% OF THE REMAINING RELEVANT DATA
# SO WE ARE DROPPING THE CORRESPONDING ROWS

cleaned_data1 = reordered_data.dropna(subset=['company_name'])
cleaned_data2 = cleaned_data1.dropna(subset=['location'])
cleaned_data3 = cleaned_data2.dropna(subset=['dma_id'])
cleaned_data4 = cleaned_data3.dropna(subset=['title'])
cleaned_data5 = cleaned_data4.dropna(subset=['tag'])
cleaned_data6 = cleaned_data5.dropna(subset=['total_yearly_compensation'])
cleaned_data7 = cleaned_data6.dropna(subset=['base_salary'])
cleaned_data8 = cleaned_data7.dropna(subset=['stock_grant_value'])
cleaned_data9 = cleaned_data8.dropna(subset=['bonus'])

cleaned_data9.isnull().sum()


cleaned_data9.head(2)

,row_number,company_name,location,city_id,dma_id,years_of_experience,years_at_company,title,tag,total_yearly_compensation,base_salary,stock_grant_value,bonus,gender,race,education,otherdetails
0,11381,INTEL,"Santa Clara, CA",7434,807.0,14.0,3.0,Hardware Engineer,Physical Design,200000.0,163000.0,10000.0,27000.0,Female,Asian,NaN,Asian
1,16214,LATCH,"New York, NY",10182,501.0,6.0,2.0,Software Engineering Manager,Full Stack,170000.0,170000.0,0.0,0.0,Male,Asian,NaN,Asian


In [29]:

# # COMBIEN DE DONNEES MANQUENT PAR TYPE? => print(data.isnull().sum())
# # + pour afficher uniquement les variables qui ont des valeurs manquantes
# #nb_na = data.isnull().sum()
# #print(nb_na[nb_na>0])
# print("Il manque le nombre de données des types suivants dans les différentes colonnes:\n")
# print(data.isnull().sum())
# print("\n")


In [30]:

# # + CORRECTION DES TYPES ERRONES :
# data['date_operation'] = pd.to_datetime(data['date_operation'], errors="coerce")
# print("\n")
# print("Après correction, les différents types de données actuels de ce dataframe sont les suivants:\n")
# print(data.dtypes)
# print("\n")



In [31]:
#Average_salaries

Business_Analyst = 103505.08
Data_Scientist= 138054.69
Hardware_Engineer = 39628.18
Human Resources = 128590.66
Management_Consultant = 136195.7 
Marketing = 140957.75
Mechanical_Engineer = 118455.1
Product_Designer =137163.59
Product_Manager = 148672.27
Recruiter = 117496.67
Sales = 118470.72
Soft Engineer =118500.0
Software Engineer = 132418.75 
Software Engineering Manager = 174204.47 
Solution Architect = 140173.51
Technical Program Manager = 156454.74

SyntaxError: invalid syntax (164866818.py, line 6)

In [ ]:
pd.